In [1]:
import pandas as pd
from recordlinkage.index import Block
from recordlinkage.compare import Compare

# Load your data
df_a = pd.DataFrame({
    'name': ['John Smith', 'Jane Doe', 'Mary Johnson'],
    'dob': ['1990-01-01', '1985-05-15', '1970-12-30']
})

df_b = pd.DataFrame({
    'name': ['John Smith', 'Jane D.', 'Mary Johnson'],
    'dob': ['1990-01-01', '1985-05-15', '1970-12-30']
})

# Create index pairs
indexer = Block('dob')
candidate_links = indexer.index(df_a, df_b)

# Compare the data
compare = Compare()
compare.string('name', 'name', method='jarowinkler', threshold=0.85, label='name')
compare.exact('dob', 'dob', label='dob')

comparison_vectors = compare.compute(candidate_links, df_a, df_b)

# Calculate frequency tables
name_frequency_a = df_a['name'].value_counts(normalize=True).to_dict()
name_frequency_b = df_b['name'].value_counts(normalize=True).to_dict()

def frequency_adjustment(row, freq_a, freq_b):
    name_a, name_b = row['name']
    return 1 - max(freq_a.get(name_a, 0), freq_b.get(name_b, 0))

comparison_vectors['name_weighted'] = comparison_vectors.apply(frequency_adjustment, axis=1, args=(name_frequency_a, name_frequency_b))

# Adjust scores based on frequency
comparison_vectors['final_score'] = comparison_vectors['name'] * comparison_vectors['name_weighted']
comparison_vectors['final_score'] += comparison_vectors['dob']

# Set a threshold and make predictions
threshold = 1.5
matches = comparison_vectors[comparison_vectors['final_score'] > threshold]

# Review the results
matched_pairs = matches.index
print(matched_pairs)


ImportError: cannot import name 'Compare' from 'recordlinkage.compare' (/home/tania/.local/lib/python3.10/site-packages/recordlinkage/compare.py)